In [1]:
import math
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


In [2]:
#Clear the previous tensorflow graph
K.clear_session()

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the results of running DE on the old model (250->50->1)
windowSize = 20
stride = 2
constRUL = 105

dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)

In [5]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
windowSizes = {1:25,2:20,3:25,4:15}

for i in range(1,5):
    
    dataset = i
    print("Computing for dataset "+str(i))
    
    #Create and compile the models
    windowSize = windowSizes[dataset]
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windowSize
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    tModel.changeModel('ModelRUL_SN_Dat'+str(shapeSN), modelRULSN)
    tModel.windowSize = windowSizes[dataset]
    tModel.changeDataset(dataset)
    tModel.loadData()
    tempScoresRMSE = np.zeros((iterations,1))
    tempScoresRHS = np.zeros((iterations,1))
    tempTime = np.zeros((iterations,1))

    for j in range(iterations):
        
        print("Iteration "+str(j+1))
        tModel.trainModel(learningRateScheduler=lrate, verbose=0)
        tModel.evaluateModel(["rhs"])

        cScores = tModel.scores
        rmse = math.sqrt(cScores['score_1'])
        rhs = cScores['rhs']
        time = tModel.trainTime
        
        tempScoresRMSE[j] = rmse
        tempScoresRHS[j] = rhs
        tempTime[j] = time

    print(stats.describe(tempScoresRMSE))
    print(stats.describe(tempScoresRHS))
    print(stats.describe(tempTime))

Computing for dataset 1
Iteration 1
100/100 [==============================] - 0s 858us/step
Iteration 2
100/100 [==============================] - 0s 84us/step
Iteration 3
100/100 [==============================] - 0s 73us/step
Iteration 4
100/100 [==============================] - 0s 72us/step
Iteration 5
100/100 [==============================] - 0s 100us/step
Iteration 6
100/100 [==============================] - 0s 63us/step
Iteration 7
100/100 [==============================] - 0s 63us/step
Iteration 8
100/100 [==============================] - 0s 80us/step
Iteration 9
100/100 [==============================] - 0s 99us/step
Iteration 10
100/100 [==============================] - 0s 73us/step
DescribeResult(nobs=10, minmax=(array([ 11.1100192]), array([ 12.3774191])), mean=array([ 11.50243171]), variance=array([ 0.12882941]), skewness=array([ 1.52082358]), kurtosis=array([ 1.6482512]))
DescribeResult(nobs=10, minmax=(array([ 165.22608084]), array([ 204.16828823])), mean=array([ 17